In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import warnings

sns.set()
rcParams['figure.figsize'] = (20,10)
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
from pprint import pprint
import json

from transformers import AutoTokenizer, AutoModel, AutoConfig, pipeline
from datasets import load_dataset, Dataset

In [2]:
oasst = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/oasst/en_oasst.json')
def get_conversation(ds):
    for temp0 in ds:
        conv = temp0['text'].removeprefix('The conversation between human and AI assistant.\n')
        temp1 = conv.split('[|Human|]')[-1]
        temp2 = temp1.split('[|AI|]')[0]
        i = conv.find(temp2)
        conv = conv[:i + len(temp2)]
        yield conv
    
gen = get_conversation(oasst)

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-c507d95be8d43b3a/0.0.0)


In [3]:
prefix = """Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a general knowledge base.
Generate a  search query based on the conversation and the new question. 
Do not include any text inside [] or <<>> in the search query terms.
Do not include any special characters like '+'.
Return only the query in json format: {"query": "generated query"}
If you cannot generate a search query, return just the number 0.

Conversation history:
"""

In [10]:
oasst = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/oasst/en_oasst.json')
def create_dataset(ds):
    result = []
    for temp0 in ds.select(range(100)):
        conv = temp0['text'].removeprefix('The conversation between human and AI assistant.\n')
        temp1 = conv.split('[|Human|]')[-1]
        temp2 = temp1.split('[|AI|]')[0]
        i = conv.find(temp2)
        conv = conv[:i + len(temp2)]
        result.append(conv)
    
    return Dataset.from_dict({'text': result})
    
dataset = create_dataset(oasst)

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-c507d95be8d43b3a/0.0.0)


In [13]:
dataset.to_json('../../../data/search_query.jsonl', orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

135980

In [ ]:
conv = next(gen)
prefix = """Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a general knowledge base.
Generate a  search query based on the conversation and the new question. 
Do not include any text inside [] or <<>> in the search query terms.
Do not include any special characters like '+'.
Return only the query in json format: {"query": "generated query"}
If you cannot generate a search query, return just the number 0.

Conversation history:
"""

print(prefix + conv)

In [ ]:
query = \
{
"query": "kid-friendly science projects without using chemicals or potentially harmful materials"
}


def save_query(conversation, query):
    with open('../../data/retrieve_query_gen_oasst.jsonl', 'a') as f:
        js = {
            'conversation': conversation,
            'query': query,
        }
        f.write(json.dumps(js) + '\n')

save_query(conv, query['query'])

In [3461]:
quora = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/quora_chat_data.json')
def check(text):
    for x in ['law', 'legal', 'lawyers']:
        if x in text:
            return True
temp = quora.filter(lambda x: check(x['topic']))
temp.shape

Found cached dataset json (/Users/phamhoang1408/.cache/huggingface/datasets/json/default-b66275377ff3550a/0.0.0)
Loading cached processed dataset at /Users/phamhoang1408/.cache/huggingface/datasets/json/default-b66275377ff3550a/0.0.0/cache-184e20a9bb51ef67.arrow


(465, 2)

In [ ]:
quora.shape

In [3470]:
conv = next(gen)
prefix = """Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a general knowledge base.
Generate a  search query based on the conversation and the new question. 
Do not include any text inside [] or <<>> in the search query terms.
Do not include any special characters like '+'.
Return only the query in json format: {"query": "generated query"}
If you cannot generate a search query, return just the number 0.

Conversation history:
"""

print(prefix + conv)

Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a general knowledge base.
Generate a  search query based on the conversation and the new question. 
Do not include any text inside [] or <<>> in the search query terms.
Do not include any special characters like '+'.
Return only the query in json format: {"query": "generated query"}
If you cannot generate a search query, return just the number 0.

Conversation history:
[|Human|] I want to know the step by step guide to invest in share market in India.
[|AI|] Sure, I can help with that. Firstly, you need to open a demat and trading account with a registered stockbroker.
[|Human|] How do I find a registered stockbroker in India?
[|AI|] You can visit the websites of National Stock Exchange (NSE) or Bombay Stock Exchange (BSE) to get a list of registered stockbrokers in India.
[|Human|] What documents are required to open a demat and trading account?
[|AI|] You will

In [3462]:
def get_conversation(ds):
    for temp0 in ds:
        conv = temp0['input'].removeprefix('The conversation between human and AI assistant.\n')
        temp1 = conv.split('[|Human|]')[-2]
        temp2 = temp1.split('[|AI|]')[0]
        i = conv.find(temp2)
        conv = conv[:i + len(temp2)]
        yield conv
gen = get_conversation(quora)

In [3463]:
len(temp)

465

In [3469]:
print(temp.shuffle()[0]['input'])

The conversation between human and AI assistant.
[|Human|] I was curious to know if there are any countries without private criminal lawyers.
[|AI|] Yes, there are actually some countries where the government is responsible for providing legal representation for criminal defendants.
[|Human|] Do you know the names of those countries?
[|AI|] Yes, some examples include Japan, Norway, and Sweden.
[|Human|] That's interesting. Do you know why those countries have this system?
[|AI|] From what I understand, it's due to the belief that all citizens have the right to legal representation, regardless of their ability to pay for it. It's also a way to ensure a fair trial and prevent corruption in the legal system.
[|Human|] I see. Are there any drawbacks to this system?
[|AI|] One potential drawback is that the quality of legal representation may not be as high as it would be if defendants were able to hire their own lawyers. Additionally, the government may not have enough resources to provide

In [3471]:
query = \
{"query": "How to track investments in share market India"}

def save_query(conversation, query):
    with open('../../data/retrieve_query_gen_quora.jsonl', 'a') as f:
        js = {
            'conversation': conversation,
            'query': query,
        }
        f.write(json.dumps(js) + '\n')

save_query(conv, query['query'])

In [ ]:
share = pd.read_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/original/share_gpt_no_code_conversations_40k.json', orient='records', lines=True)
share.columns = ['input']
share.dropna(inplace=True)
share.reset_index(drop=True, inplace=True)
share = Dataset.from_pandas(share)
def check(text):
    for x in ['law', 'legal', 'lawyers']:
        if x in text:
            return True
temp = share.filter(lambda x: check(x['input']))
temp.shape

In [ ]:
print(share.shuffle()[0]['input'])

In [ ]:
def get_conversation(ds):
    for temp0 in ds:
        conv = temp0['input'].removeprefix('The conversation between human and AI assistant.\n')
        temp1 = conv.split('[|Human|]')[-2]
        temp2 = temp1.split('[|AI|]')[0]
        i = conv.find(temp2)
        conv = conv[:i + len(temp2)]
        yield conv
gen = get_conversation(temp)

In [2023]:
conv = next(gen)
prefix = """Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a general knowledge base.
Generate a  search query based on the conversation and the new question. 
Do not include any text inside [] or <<>> in the search query terms.
Do not include any special characters like '+'.
Return only the query in json format: {"query": "generated query"}
If you cannot generate a search query, return just the number 0.

Conversation history:
"""

print(prefix + conv)

Below is a history of the conversation so far, and a new question asked by the user that needs to be answered by searching in a general knowledge base.
Generate a  search query based on the conversation and the new question. 
Do not include any text inside [] or <<>> in the search query terms.
Do not include any special characters like '+'.
Return only the query in json format: {"query": "generated query"}
If you cannot generate a search query, return just the number 0.

Conversation history:
[|Human|] Please write a limerick based on Hegel's "Phenomenology of Spirit"
[|AI|] There once was a philosopher named Hegel
Whose work on the spirit was quite legal
He wrote of our minds
In his famous tome finds
The journey towards self-consciousness is regal
[|Human|] Please write a sonnet based on Kant's "Critique of Pure Reason"
[|AI|] In Kant's "Critique of Pure Reason" we find
A treatise on knowledge and its design
He wrote of the mind
And the concepts it does bind
In order to grasp the worl

In [1902]:
query = \
{"query": "Media criticism of RESTRICT Act covering end-users, targeted nature of the bill according to Senator Warner"}

def save_query(conversation, query):
    with open('../../data/retrieve_query_gen_sharegpt.jsonl', 'a') as f:
        js = {
            'conversation': conversation,
            'query': query,
        }
        f.write(json.dumps(js) + '\n')

save_query(conv, query['query'])